In [ ]:
from VwPipeline.Core import Workspace
from VwPipeline import Logger
from VwPipeline.Vw import Vw, VwInput
from VwPipeline.VwSweep import VwSweep
from VwPipeline import VwOptsGrid

logger = Logger.console_logger(0, 'INFO')
context = Workspace(r'C:\data\sweep\ws', logger)

vw_path = 'vw'

vw = Vw(vw_path, context)

sweep = VwSweep(vw, input_mode=VwInput.cache)

In [ ]:
input_d = [r'C:\data\sweep\1.json', r'C:\data\sweep\2.json']

base = {'#base': '--ccb_explore_adf --epsilon 0 --dsjson -b 20'}

Caching step

In [ ]:
opts_out = ['--cache_file']
cache_result = vw.test(input_d, base, opts_out)
logger.info('Done')
input_c = [i['--cache_file'] for i in cache_result.Populated]
input_c

Hyper parameters for single iteration

In [ ]:
base = {'#base': '--ccb_explore_adf --epsilon 0 --dsjson --power_t 0 --cb_type ips --save_resume', "#int": '-q bc'}

from VwPipeline.VwOptsGrid import dimension, product, Grid, Configuration
import pandas as pd

hyper_points = product(
    [base],
    dimension('-l', [1e-1, 1e-2]),
    dimension('--l1', [1e-09, 1e-07])
)

Iterations - list version

In [ ]:
results = sweep.iteration(input_c, hyper_points)
results = pd.DataFrame(map(lambda r: {'Loss': r.Loss, 'Args': r.Opts}, results)).sort_values('Loss')
results

In [ ]:
top = results.head(2)['Args'].to_list()
top

In [ ]:
new_points = dimension('#coin', ['', '--coin'])
new_points = product(top, new_points)
new_points

In [ ]:
results = sweep.iteration(input_c, new_points)
results = pd.DataFrame(map(lambda r: {'Loss': r.Loss, 'Args': r.Opts, 'Name': r.Name}, results)).sort_values('Loss')
results

In [ ]:
predict_opts = {'#base': '--ccb_explore_adf --epsilon 0.2 --compressed --dsjson --save_resume --preserve_performance_counters'}
predict_args = [VwOptsGrid.apply(p, predict_opts) for p in results['Args']]
sweep.predict(input_c, predict_args)

Pandas version

In [ ]:
hp_df = pd.DataFrame({'Args': hyper_points})

In [ ]:
results = sweep.iteration_df(input_c, hp_df)
results

In [ ]:
top = results.head(2)['Args'].values
new_points = dimension('#coin', ['', '--coin'])
new_points = pd.DataFrame({'Args': product(top, new_points)})
new_points

In [ ]:
results = sweep.iteration_df(input_c, new_points)
results

In [ ]:
predict_opts = {'#base': '--ccb_explore_adf --epsilon 0.2 --compressed --dsjson --save_resume --preserve_performance_counters'}
results['PredictArgs'] = [VwOptsGrid.apply(p, predict_opts) for p in results['Args']]
results

In [ ]:
predict_table = sweep.predict_df(input_c, results)
predict_table

In [ ]:
#todo: feed predict_table into dashboard build